In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit

from threading import Thread, Lock

/Users/danilo/.conda/envs/bcb/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


## Dataset Configuration

In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

C = d_clinical.values

y_d = outputs["dement_fail"].values
y_c = outputs["cvd_fail"].values

In [3]:
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y_d))

C_ = C[tr_idx]
C_test = C[ts_idx]

y_d_ = y_d[tr_idx]
y_d_test = y_d[ts_idx]

y_c_ = y_c[tr_idx]
y_c_test = y_c[ts_idx]

In [4]:
ds_list = [C_]
ds_test = [C_test]
ds_names = ['clinic']

## Kernel Definition

In [5]:
kernel_names = ['linear', 'laplacian']
kernel_type = [{'linear':[1], 'laplacian':[0.2, 0.4]},
               {'linear':[1], 'laplacian':[0.3, 0.6]}]

## Other shared parameters initialization

In [6]:
estimator = ca.centeredKernelAlignment
lock_dementia = Lock()
lock_cardio = Lock()
valid_fold = 3
threads = []
pen_params = [0.7, 1]

## Thread

In [7]:
def child(sampler,estimator,ds_list,ds_test,y_d_,y_d_test,y_c_,y_c_test,lock_dementia,lock_cardio,valid_fold,exclusion_list,verbose,approach):
    """
    #DEMENTIA
    
    result1 = sampler.sample(kernel_type, estimator, ds_list, y_d_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names)
    result1.performancesFeatures(fileToWrite = 'results_temp/Dementia_train.txt', header = '\nDementia Laplacian - Gaussian\n' + approach + '\n', lock = lock_dementia)
    ut.testConfigurations(estimator, y_d_, y_d_test, w_list, ds_list, ds_test, kernel_names, lamb_list, sparsity, 'classification', lock_dementia, fileToWrite = 'results_temp/Dementia_test.txt', header = 'Dementia Linear, Laplacian, Sigmoid, Polynomial, Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    """   
    # CARDIO
    
    result1 = sampler.sample(kernel_type, estimator, ds_list, y_c_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names)
    result1.performancesFeatures(fileToWrite = 'results_temp/Cardio_train.txt', header = '\nCardio Laplacian - Gaussian\n' + approach + '\n', lock = lock_cardio)
    ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names, lamb_list, sparsity, 'classification', lock_cardio, fileToWrite = 'results_temp/Cardio_test.txt', header = 'Cardio Linear, Laplacian, Sigmoid, Polynomial, Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)


## L2, Centering, Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - Normalizing'))
t.start()
threads.append(t)

## L2, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Normalizing - K Normalizing'))
t.start()
threads.append(t)

## L2, Centering, K-Normalization

In [8]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [9]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - K Normalizing'))
t.start()
threads.append(t)

[13472.23557933  2895.17465679]
[13472.23557933  3324.47038626]
[13404.77546094  2761.88153304]
[13404.77546094  3156.36978628]
[11281.37188099  2477.86134003]
[11281.37188099  2836.1368946 ]
[28285.78680276  6556.05912033]
[13472.23557933  3269.81095549]
[13472.23557933  3016.40160492]
[13404.77546094  3112.11766127]
[13404.77546094  2849.71930814]
[11281.37188099  2792.69092134]
[11281.37188099  2575.94926057]
[28285.78680276  6518.37176061]
[14313.15935555  3205.86216976]
[14313.15935555  3645.90282029]
[14256.60112936  3175.3809935 ]
[14256.60112936  3609.54174067]
[12979.95441594  2701.11373854]
[12979.95441594  3085.52542783]
[30929.14265492  7353.61529904]
[14313.15935555  3603.47441202]
[14313.15935555  3276.46685497]
[14256.60112936  3569.10684837]
[14256.60112936  3237.02170878]
[12979.95441594  3041.16615017]
[12979.95441594  2797.24267061]
[30929.14265492  7327.0636362 ]
[14258.06962181  3098.81512794]
[14258.06962181  3505.06868135]
[17245.41842687  3681.6894864 ]
[17245.4

## L2, Centering, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - Normalizing - K Normalizing'))
t.start()
threads.append(t)

In [10]:
for t in threads:
    t.join()

print("Operations completed")
threads = []

[15055.84810492  3211.92857148]
[15055.84810492  3670.89866707]
[14271.12372564  2925.00189662]
[14271.12372564  3361.33057529]
[17470.94769747  3685.63404907]
[17470.94769747  4117.73367458]
[34803.21733249  8004.16550842]
[15055.84810492  3619.90291175]
[15055.84810492  3309.68316578]
[14271.12372564  3305.98326177]
[14271.12372564  3044.32010807]
[17470.94769747  4106.85165945]
[17470.94769747  3630.32395654]
[34803.21733249  7038.26080711]
[12388.64040637  2899.72777633]
[12388.64040637  3331.94275801]
[13783.44973277  3122.13101942]
[13783.44973277  3513.94784827]
[13089.13244976  2706.99613758]
[13089.13244976  3127.92154145]
[29041.76498319  6257.1631252 ]
[12388.64040637  3275.65234126]
[12388.64040637  3028.21138536]
[13783.44973277  3491.56918753]
[13783.44973277  3123.06745377]
[13089.13244976  3067.46265136]
[13089.13244976  2852.38691644]
[29041.76498319  6989.70493851]
[15391.94432993  3341.90371862]
[15391.94432993  3809.06236607]
[13855.7049197   2900.58418372]
[13855.7

## L1, Centering, Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - Normalizing'))
t.start()
threads.append(t)

## L1, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Normalizing - K Normalizing'))
t.start()
threads.append(t)

## L1, Centering, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - K Normalizing'))
t.start()
threads.append(t)

## L1, Centering, Normalization, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - Normalizing - K Normalizing'))
t.start()
threads.append(t)

In [ ]:
for t in threads:
    t.join()

print("Operations completed")
threads = []